# 06. 토픽 모델링(Topic Modeling) - 20 뉴스 그룹

p506(525)~

In [1]:
import warnings
warnings.filterwarnings('ignore')

<br>

## 6.1 토픽 모델링 (Topic Modeling)

- 문서 집합에 숨어 있는 주제를 찾아내는 것
- 머신러닝 기반의 토픽 모델링을 적용해 숨어 있는 중요 주제를 효과적으로 찾아낼 수 있다.
- 사람이 수행하는 토픽 모델링 : 더 함축적인 의미로 문장을 요약
- 머신러닝 기반 토픽 모델링 : 숨겨진 주제를 효과적으로 표현할 수 있는 중심 단어를 함축적으로 추출

<br>

## 6.2 머신러닝 기반  토픽 모델링 기법

- LSA (Latent Semantic Analysis)
- LDA (Latent Dirichlet Allocation)
  - 차원 축소의 LDA(Linear Discriminant Analysis)와 약어만 같은 뿐 서로 다른 알고리즘이다.

<br>

## 6.3 20 뉴스그룹 데이터에 토픽 모델링 적용

- 20 뉴스그룹은 다음과 같이 20가지 주제를 가진 뉴스그룹 데이터를 가지고 있음

```python
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles',
 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space',
 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc',
 'talk.religion.misc']
```

- 이 중 모토사이클, 야구, 그래픽스, 윈도우, 중동, 기독교, 전자공학, 의학의 8개 주제를 추출하고 이들 텍스트에 LDA 기반의 토픽 모델링을 적요

<br>

### 6.3.1 데이터 로드 및 Count 기반 피처 벡터화

- 사이킷런은 LDA(Latent Dirichler Allocation) 기반의 토픽 모델링을 `LatentDirichletAllocation` 클래스로 제공

- LDA 토픽 모델링을 위해 `fetch_20newsgroups()` API는 `categories` 파라미터를 통해 필요한 주제만 필터링해 추출
- 추출된 텍스트를 Count 기반으로 벡터화 변환
  - LDA는 Count 기반의 벡터화만 사용
  - `max_features=1000` 으로 word 피처의 개수를 제한
  - `ngram_range`는 `(1,2)` 로 설정

In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 모토사이클, 야구, 그래픽스, 윈도우즈, 중동, 기독교, 의학, 우주 주제를 추출. 
cats = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x',
        'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med'  ]

# 위에서 cats 변수로 기재된 카테고리만 추출. fetch_20newsgroups()의 categories에 cats 입력
news_df = fetch_20newsgroups(subset='all', remove={'headers', 'footers', 'quotes'},
                            categories=cats, random_state=0)

# LDA는 Count  기반의 벡터화만 적용
count_vect = CountVectorizer(max_df=0.95, max_features=1000, min_df=2, stop_words='english',
                             ngram_range=(1,2))
feat_vect = count_vect.fit_transform(news_df.data)
print('CountVectorizer Shape: ', feat_vect.shape)

CountVectorizer Shape:  (7862, 1000)


- CountVectorizer 객체 변수인 `feat_vect`는 모두 7862개의 문서가 1000개의 피처로 구성된 행렬 데이터

<br>

### 6.3.2 LDA 토픽 모델링 수행

- 토픽의 개수는 위의 뉴스그룹에서 추출한 주제와 동일한 8개로 설정
- `LatentDirichletAllocation` 클래스의 `n_components` 파라미터를 이용해 토픽 개수 조정

In [4]:
lda = LatentDirichletAllocation(n_components=8, random_state=0)
lda.fit(feat_vect)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

<br>

### 6.3.3 `components_` 속성값 확인

- `latentDirichletAllocation.fit(데이터 세트)`을 수행하면 `latentDirichletAllocation` 객체는 `components_` 속성값을 가지게 된다.
- `components_`는 개별 토픽별로 각 word 피처가 얼마나 많이 그 토픽에 할당됐는 지에 대한 수치를 가지고 있음
- 높은 값일수록 해당 word 피처는 그 토픽의 중심 word가 된다.

- `components_`의 형태와 속성값을 확인

In [5]:
print(lda.components_.shape)
lda.components_

(8, 1000)


array([[3.60992018e+01, 1.35626798e+02, 2.15751867e+01, ...,
        3.02911688e+01, 8.66830093e+01, 6.79285199e+01],
       [1.25199920e-01, 1.44401815e+01, 1.25045596e-01, ...,
        1.81506995e+02, 1.25097844e-01, 9.39593286e+01],
       [3.34762663e+02, 1.25176265e-01, 1.46743299e+02, ...,
        1.25105772e-01, 3.63689741e+01, 1.25025218e-01],
       ...,
       [3.60204965e+01, 2.08640688e+01, 4.29606813e+00, ...,
        1.45056650e+01, 8.33854413e+00, 1.55690009e+01],
       [1.25128711e-01, 1.25247756e-01, 1.25005143e-01, ...,
        9.17278769e+01, 1.25177668e-01, 3.74575887e+01],
       [5.49258690e+01, 4.47009532e+00, 9.88524814e+00, ...,
        4.87048440e+01, 1.25034678e-01, 1.25074632e-01]])

- `components_`는 `array[8, 4000]`으로 구성돼 있음
- 8개의 토픽별로 1000개의 word 피처가 해당 토픽별로 연관도 값을 가지고 있음
- ex) `components_` array의 0번째 row, 10번째 col에 있는 값  
$\rightarrow$ Topic #0에 대해서 피처 벡터화된 행렬에서 10번째 컬럼에 해당하는 피처가 Topic #0에 연관되는 수치 값을 가지고 있음

- `lda_model.components_` 값만으로는 각 토픽별 word 연관도를 보기가 어려움
- `display_topics()` 함수를 만들어서 각 토픽별로 연관도가 높은 순으로 Word를 나열

In [6]:
def display_topics(model, feature_names, no_top_words):
    
    for topic_index, topic in enumerate(model.components_):
        print('Topic #', topic_index)
        
        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array 인덱스를 반환
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes = topic_word_indexes[:no_top_words]
        
        # top_indexes 대상인 인덱스별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])
        print(feature_concat)
        
# CountVectorizer 객체 내의 전체 word의 명칭을 get_features_names() 를 통해 추출
feature_names = count_vect.get_feature_names()

# 토픽별 가장 연관도가 높은 word를 15개만 추출
display_topics(lda, feature_names, 15)

Topic # 0
year 10 game medical health team 12 20 disease cancer 1993 games years patients good
Topic # 1
don just like know people said think time ve didn right going say ll way
Topic # 2
image file jpeg program gif images output format files color entry 00 use bit 03
Topic # 3
like know don think use does just good time book read information people used post
Topic # 4
armenian israel armenians jews turkish people israeli jewish government war dos dos turkey arab armenia 000
Topic # 5
edu com available graphics ftp data pub motif mail widget software mit information version sun
Topic # 6
god people jesus church believe christ does christian say think christians bible faith sin life
Topic # 7
use dos thanks windows using window does display help like problem server need know run
